In [18]:
import pandas as pd
import dashscope
import logging
import os

logging.basicConfig(format='%(asctime)s - %(filename)s[line:%(lineno)d] - %(levelname)s: %(message)s',
                    level=logging.INFO,
                    filename='llm_model_log.log',
                    filemode='a')

In [19]:
import time
def dashscope_predict_one(prompt):    
    time.sleep(2)
    messages = [
        {'role': 'system', 'content': '你是一名立场检测专家。'},
        {'role': 'user', 'content': prompt}
        ]
    try:
        response = dashscope.Generation.call(
            # 若没有配置环境变量，请用百炼API Key将下行替换为：api_key="sk-xxx",
            api_key='sk-b8717756fc0f4b7290f3aa7b931c448e',
            model=llm_model, # 模型列表：https://help.aliyun.com/zh/model-studio/getting-started/models
            messages=messages,
            result_format='message',
            max_tokens=10
            )
        result = response.get("output").get("choices")[0].get("message").get("content").strip()
        logging.info(response)
        logging.info(result)
    except Exception as e:
        result = "ERROR"
        # print(response)
        # print(f"Error processing prompt: {response}\n{e}\n{prompt}")
        
        logging.error(response)
        logging.error(f"Error processing prompt: {prompt}\n{e}")
        # time.sleep(300)


    return result

In [20]:
# 选择模型
# choice_list = ['qwen2-7b-instruct','qwen2-72b-instruct','llama3-8b-instruct','chatglm3-6b','llama3-70b-instruct']
choice_list = ['llama3-8b-instruct','chatglm3-6b','llama3-70b-instruct']
for model in choice_list:
    llm_model = model
    print(llm_model)
    # log file

    logging.info(llm_model+'*'*30)
    # load data
    llm_result_file = f'{llm_model}.pkl'
    if os.path.exists(llm_result_file):
        df = pd.read_pickle(llm_result_file)
    else:
        df = pd.read_pickle('test_data_unlable.pkl')
    
    # 遍历DataFrame更新'PREDICTION'列
    for index, row in df.iterrows():
        if row['PREDICTION'] in ["NULL", "ERROR"]:
            print(row['PROMPT'])
            # 调用dashscope_predict_one函数进行预测
            logging.info(f'index:{index}')
            prediction = dashscope_predict_one(row['PROMPT'])
            # prediction = '反对'
            # 更新'PREDICTION'列
            df.at[index, 'PREDICTION'] = prediction
    df.to_pickle(llm_result_file)

llama3-8b-instruct
chatglm3-6b
llama3-70b-instruct
